Модель процесса обслуживания покупателей на кассах в магазине

In [ ]:
# -*- coding: utf-8 -*-
# Импорт среды SimPy
import simpy
# Для генерации случайных чисел
import random
# В этой модели полагаем, что единица модельного времени
# соответствует 1 секунде реального времени
# Длительность покупки\получения услуги
MAX_SERVICE_DURATION = 20 * 60
# т.е. максимальная длительность обслуживания 20 мин
MODE_SERVICE_DURATION = 7 * 60
# индивидуальное время обслуживания треугольно распределенной
# случайной величине на интервале [30, 20*60, 7*60]

# макс.интервал до появления нового покупателя
ARRIV_INTER = 3 * 60
# время появления нового покупателя равномерно распределенная
# случайная величина на интервале [0, 4 * 60]

# Время работы магазина (10 часов)
# После 10 часов работы магазин закрывают и обслуживают только оставшихся, 
# если таковые имеются
CONSUMER_TIME = 3600 * 10

# ------------ параметры для статистики --------
quelog = [] # журнал длины очереди
tservlog =[] # журнал пребывания в системе

Класс описания поведения клиента

In [ ]:
# Класс - покупатель магазина - логический блок-процессор
class Client(object):
    def __init__(self, env, res, name='client'):
        self.name = name  # Имя покупателя, чтобы их различать
        self.env = env  # Среда моделирования
        self.res = res  # используемый при моделировании ресурс,- касса

    def run(self):
        # ссылка на глобальные счетчики статистики
        # для графиков после моделирования
        global  quelog, tservlog, rescnt
        # человек пришел и сразу встал в очередь: она увеличилась на 1
        # Запомним время, чтобы посчитать потом время пребывания в очереди
        timeq = self.env.now
        print(f"Привет! Это {self.name} и я прибыл в магазин в {timeq} (сек)")
        # Запрос свободной кассы
        with self.res.request() as req:
            # Нет свободной кассы? в очередь
            yield req
            # Свободная касса появилась!
            # Человек поступает на обслуживание и очередь уменьшается на 1
            # запомним текущую длину очереди
            # запомним текущее время события
            # переменная (wait-timeq) - время, проведенное в очереди
            wait = self.env.now
            quelog.append((wait, len(self.res.queue)))
            # время обслуживания - случайное число, генерируем его
            serving_duration = int(random.triangular(30, MAX_SERVICE_DURATION, MODE_SERVICE_DURATION))
            # обслуживаемся в кассе!
            yield self.env.timeout(serving_duration)
            # Обслужились и освободили кассу
            print(f"Я {self.name}, обслуживался {serving_duration} сек, ждал в очереди {wait-timeq} сек")
            # Запомним время проведенное в магазине
            # Запомним текущее время события - клиент ушел
            tservlog.append((self.env.now, self.env.now-timeq))
            print(f"Меня обслужили и сейчас (время={self.env.now}) я ушёл.")

Подготовка модельной среды и объявление функции-генератора клиентов

In [ ]:
# Инициализация среды моделирования
env = simpy.Environment()

# Ресурс обслуживания; в данном случае - capacity - число касс,
# которые обслуживают покупателей
cashier = simpy.Resource(env, capacity=6)

# Источник покупателей предполагает, что покупатели приходят 
# в течение 10 часов от начала работы магазина
def source_men(env):
    ind = 0
    while env.now < (CONSUMER_TIME - ARRIV_INTER): # покупатели приходят 10 часов
        ind += 1
        yield env.timeout(random.randint(0, ARRIV_INTER))
        man = Client(env, cashier, name=f'клиент{ind}')
        env.process(man.run())

Настройка параметров модели и запуск прогона

In [ ]:
# инициализация датчика случайных чисел
#random.seed(13971)
# инициализация журналов статистики
quelog,tservlog = [],[]

# Добавляем процесс прихода покупателей в модель
env.process(source_men(env))

# Запускаем процесс моделирования, полагая, что 
# 1 ед.модельного времени = 1 секунда реального времени;
# процесс моделирования составляет 12 часов; 
env.run(until=12 * 60 * 60)

Подготовка статистики и вывод графиков

In [ ]:
try: 
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()
    # График длины очереди quelog
    ax.step([p[0] for p in quelog], [p[1] for p in quelog], label='очередь')
    ax.set_title('Клиентов в очереди')
    ax.set_xlabel('Время, сек')
    ax.set_ylabel('длина очереди, #')
    fig, bx = plt.subplots()
    # График времени пребывания в системе tservlog
    bx.scatter([p[0] for p in tservlog], [p[1] for p in tservlog], c='g', marker='x')
    bx.set_title(f'Время пребывания_')
    bx.set_xlabel('Время, сек')
    bx.set_ylabel('Пребывание, сек')
    plt.show()
# --------------------------------------------------------------------------------------
except ImportError:
    print('without matplotlib!')